In [ ]:
import pandas as pd

In [ ]:
path = 'cleanodibowling.csv'
df = pd.read_csv(path)
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        2652 non-null   object 
 1   Overs       2652 non-null   int64  
 2   Mdns        2652 non-null   int64  
 3   Runs        2652 non-null   int64  
 4   Wkts        2652 non-null   int64  
 5   Econ        2652 non-null   float64
 6   Opposition  2652 non-null   object 
 7   Ground      2652 non-null   object 
 8   Date        2652 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 186.6+ KB


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
df['Days_Since_Match'] = (df['Date'].max() - df['Date']).dt.days
df.to_csv('cleanodibowling.csv', index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['Opposition_Encoded'] = label_encoder.fit_transform(df['Opposition'])
df['Ground_Encoded'] = label_encoder.fit_transform(df['Ground'])

print(df[['Opposition', 'Opposition_Encoded', 'Ground', 'Ground_Encoded']].head())

df.to_csv('cleanodibowling.csv', index=False)

    Opposition  Opposition_Encoded      Ground  Ground_Encoded
0  West Indies                  16  Basseterre               3
1  West Indies                  16  Basseterre               3
2  West Indies                  16  Basseterre               3
3  West Indies                  16  Basseterre               3
4  West Indies                  16  Basseterre               3


In [ ]:
X = df[['Overs', 'Mdns', 'Runs', 'Econ', 'Days_Since_Match', 'Opposition_Encoded', 'Ground_Encoded']]
y = df['Wkts']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

Training set: (2121, 7), Test set: (531, 7)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

train_score = rf_model.score(X_train, y_train)  # R-squared score on training set
test_score = rf_model.score(X_test, y_test)  # R-squared score on testing set

print(f"Training Score (R^2): {train_score}")
print(f"Test Score (R^2): {test_score}")


Training Score (R^2): 0.8746671578585183
Test Score (R^2): 0.17213533089143884


In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 1.0791387327094837


In [ ]:
df['Predicted_Wkts'] = rf_model.predict(X)

top_10_bowlers = df[['Name', 'Predicted_Wkts']].sort_values(by='Predicted_Wkts', ascending=False).head(10)

print(top_10_bowlers)


                   Name  Predicted_Wkts
828   Mustafizur Rahman           4.335
1259       Abdur Razzak           4.250
200        Hasan Mahmud           4.110
1050       Ziaur Rahman           4.080
1435       Abdur Razzak           4.080
1656        Farhad Reza           4.080
276        Taskin Ahmed           4.040
329     Shakib Al Hasan           4.020
238     Shakib Al Hasan           3.930
1841       Abdur Razzak           3.920


In [ ]:
df['Adjusted_Recency'] = df['Days_Since_Match'].max() - df['Days_Since_Match']

print(df[['Name', 'Days_Since_Match', 'Adjusted_Recency']].head())


                 Name  Days_Since_Match  Adjusted_Recency
0        Afif Hossain                 0             13191
1      Shoriful Islam                 0             13191
2  Tanzim Hasan Sakib                 0             13191
3  Mehidy Hasan Miraz                 0             13191
4      Rishad Hossain                 0             13191


In [ ]:
import numpy as np

df['Log_Recency'] = np.log(df['Days_Since_Match'] + 1)

print(df[['Name', 'Days_Since_Match', 'Log_Recency']].head())


                 Name  Days_Since_Match  Log_Recency
0        Afif Hossain                 0          0.0
1      Shoriful Islam                 0          0.0
2  Tanzim Hasan Sakib                 0          0.0
3  Mehidy Hasan Miraz                 0          0.0
4      Rishad Hossain                 0          0.0


In [ ]:
X = df[['Overs', 'Mdns', 'Runs', 'Econ', 'Adjusted_Recency', 'Opposition_Encoded', 'Ground_Encoded']]

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)


In [ ]:
X = df[['Overs', 'Mdns', 'Runs', 'Econ', 'Adjusted_Recency', 'Opposition_Encoded', 'Ground_Encoded']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

df['Predicted_Wkts'] = rf_model.predict(X)

top_10_bowlers = df[['Name', 'Predicted_Wkts']].sort_values(by='Predicted_Wkts', ascending=False).head(10)
print(top_10_bowlers)

                   Name  Predicted_Wkts
828   Mustafizur Rahman            4.36
1259       Abdur Razzak            4.32
200        Hasan Mahmud            4.15
1050       Ziaur Rahman            4.12
276        Taskin Ahmed            4.04
1656        Farhad Reza            4.02
1435       Abdur Razzak            4.02
238     Shakib Al Hasan            3.97
1841       Abdur Razzak            3.95
329     Shakib Al Hasan            3.92


In [ ]:
df['Predicted_Wkts'] = rf_model.predict(X)

print(df[['Name', 'Predicted_Wkts']].head(10))

df.to_csv('cleanodibowling.csv', index=False)

                 Name  Predicted_Wkts
0        Afif Hossain        0.940000
1      Shoriful Islam        0.200000
2  Tanzim Hasan Sakib        0.180000
3  Mehidy Hasan Miraz        0.350000
4      Rishad Hossain        0.885000
5          Nahid Rana        1.131667
6  Mehidy Hasan Miraz        0.888333
7  Tanzim Hasan Sakib        1.196667
8        Taskin Ahmed        0.560000
9          Nahid Rana        1.030000


In [ ]:
player_predicted_wickets = df.groupby('Name')['Predicted_Wkts'].mean()  # You can use sum() instead of mean() if desired

player_predicted_wickets = player_predicted_wickets.reset_index()

print(player_predicted_wickets.head())


           Name  Predicted_Wkts
0  Abdur Razzak        1.390961
1     Abu Hider        1.625000
2     Abu Jayed        2.105000
3    Abul Hasan        0.328571
4  Afif Hossain        0.450000


In [ ]:
top_10_players = player_predicted_wickets.sort_values(by='Predicted_Wkts', ascending=False).head(10)

print(top_10_players)


                 Name  Predicted_Wkts
2           Abu Jayed        2.105000
19     Ebadot Hossain        1.647727
1           Abu Hider        1.625000
96     Sunzamul Islam        1.611111
98       Taijul Islam        1.583150
95       Subashis Roy        1.540417
65  Mustafizur Rahman        1.479750
14       Arafat Sunny        1.462500
90     Shoriful Islam        1.419583
0        Abdur Razzak        1.390961


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

decay_factor = np.exp(-df['Days_Since_Match'] / 365)  # Adjust 365 to fine-tune decay speed

df['Decay_Factor'] = decay_factor

X = df[['Overs', 'Mdns', 'Runs', 'Econ', 'Decay_Factor', 'Opposition_Encoded', 'Ground_Encoded']]
y = df['Wkts']  # Target variable: Number of wickets taken

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

df['Predicted_Wkts'] = rf_model.predict(X)

df['Weighted_Predicted_Wkts'] = df['Predicted_Wkts'] * df['Decay_Factor']

player_weighted_predicted_wickets = df.groupby('Name')['Weighted_Predicted_Wkts'].sum().reset_index()

top_10_players = player_weighted_predicted_wickets.sort_values(by='Weighted_Predicted_Wkts', ascending=False).head(10)

print(top_10_players)


                   Name  Weighted_Predicted_Wkts
50   Mehidy Hasan Miraz                17.279053
90       Shoriful Islam                15.638452
65    Mustafizur Rahman                15.086418
105        Taskin Ahmed                13.498616
85      Shakib Al Hasan                 9.003395
71          Nasum Ahmed                 7.078953
33         Hasan Mahmud                 5.604321
102  Tanzim Hasan Sakib                 5.433056
67           Nahid Rana                 4.115506
76       Rishad Hossain                 3.632862


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


y_pred = rf_model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")


Mean Absolute Error (MAE): 0.8068021552626072
Mean Squared Error (MSE): 1.1074252531968567
Root Mean Squared Error (RMSE): 1.0523427451153244
R-squared (R²): 0.15043523783230506


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor()
}

model_results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    model_results[model_name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2
    }

results_df = pd.DataFrame(model_results).T
print(results_df)


                                  MAE       MSE      RMSE        R²
Linear Regression            0.774257  1.014026  1.006989  0.222086
Decision Tree Regressor      1.064972  2.262241  1.504075 -0.735485
Gradient Boosting Regressor  0.769998  1.016966  1.008447  0.219831
SVR                          0.757544  1.094469  1.046168  0.160375
K-Nearest Neighbors          0.838795  1.215292  1.102403  0.067685


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor()
}

model_results = {}

target_range = y_train.max() - y_train.min()

for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    mae_percentage = (mae / target_range) * 100
    mse_percentage = (mse / target_range**2) * 100
    rmse_percentage = (rmse / target_range) * 100
    r2_percentage = r2 * 100

    model_results[model_name] = {
        'MAE (%)': mae_percentage,
        'MSE (%)': mse_percentage,
        'RMSE (%)': rmse_percentage,
        'R² (%)': r2_percentage
    }

results_df = pd.DataFrame(model_results).T
print(results_df)


                               MAE (%)   MSE (%)   RMSE (%)     R² (%)
Linear Regression            12.904290  2.816740  16.783146  22.208645
Decision Tree Regressor      17.749529  6.284003  25.067914 -73.548533
Gradient Boosting Regressor  12.833306  2.824905  16.807454  21.983144
SVR                          12.625730  3.040190  17.436142  16.037504
K-Nearest Neighbors          13.979912  3.375811  18.373380   6.768500


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime


df['Date'] = pd.to_datetime(df['Date'])
df['days_since_match'] = (datetime.now() - df['Date']).dt.days

df_aggregated = df.groupby('Name').agg(
    total_wickets=('Wkts', 'sum'),
    total_overs=('Overs', 'sum'),
    mean_econ=('Econ', 'mean'),
    mean_mdns=('Mdns', 'mean'),
    total_runs=('Runs', 'sum'),
    ground_list=('Ground', 'unique'),
    opposition_list=('Opposition', 'unique'),
    avg_days_since_match=('days_since_match', 'mean')
).reset_index()

df_aggregated['Econ_Wkts_Interaction'] = df_aggregated['mean_econ'] * df_aggregated['total_wickets']
df_aggregated['Econ_Wkts_Ratio'] = df_aggregated['mean_econ'] / (df_aggregated['total_wickets'] + 1)  # Avoid division by zero
df_aggregated['Econ_Wkts_Difference'] = df_aggregated['mean_econ'] - df_aggregated['total_wickets']

# X = df_aggregated[['mean_econ', 'mean_mdns', 'total_overs', 'Econ_Wkts_Interaction', 'Econ_Wkts_Ratio', 'Econ_Wkts_Difference']]
# model = GradientBoostingRegressor()  # Your trained model

df_aggregated['Predicted_Wkts'] = df_aggregated['total_wickets']  # Replace with actual model predictions

def apply_time_decay(row, lambda_decay=0.01):
    decay_score = row['Predicted_Wkts'] * np.exp(-lambda_decay * row['avg_days_since_match'])
    return decay_score

df_aggregated['Decay_Score'] = df_aggregated.apply(apply_time_decay, axis=1)

def boost_for_venue_opposition(row, venue_input, opposition_input):
    score = row['Decay_Score']  # Start with the decay-adjusted score

    if venue_input in row['ground_list']:
        score += 0.5

    if opposition_input in row['opposition_list']:
        score += 0.3

    return score

opposition_input = 'West Indies'
venue_input = 'Basseterre'

df_aggregated['Adjusted_Score'] = df_aggregated.apply(boost_for_venue_opposition, axis=1, venue_input=venue_input, opposition_input=opposition_input)


print(top_10_bowlers)

                   Name  Predicted_Wkts
828   Mustafizur Rahman            4.36
1259       Abdur Razzak            4.32
200        Hasan Mahmud            4.15
1050       Ziaur Rahman            4.12
276        Taskin Ahmed            4.04
1656        Farhad Reza            4.02
1435       Abdur Razzak            4.02
238     Shakib Al Hasan            3.97
1841       Abdur Razzak            3.95
329     Shakib Al Hasan            3.92
